In [62]:
# imports
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import os

In [2]:
#users = interactions_emb
interactions = pd.read_csv("embeddings/users_filtered_final.csv") #document with user interactions
interactions.head()

,User,ID,Interactions_emb
0,U244,N17157 N38621 N35022 N50578 N264 N9120 N23907 ...,"[-0.005149974951877837, -0.013250857458654631,..."
1,U68369,N19381 N54536,"[0.0025621717686590273, 0.004183989018201828, ..."
2,U50236,N4020 N44292 N50292 N40772 N57737 N33969 N4054...,"[-0.010138329240492436, -0.01179651383115145, ..."
3,U77060,N23105 N41375,"[-0.005568941123783588, -0.025914330035448074,..."
4,U5596,N459 N56253 N62931 N55846 N29849 N45729 N62834...,"[-0.012533644353970886, -0.011675744312297967,..."


In [3]:
#load the data with news articles
news = pd.read_csv("embeddings/news_emb_final.csv") #document with news content
news.head()

,ID,Category,SubCategory,Content,Content_emb
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","[0.005885085556656122, -0.007782096974551678, ..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat These seemingly ...,"[-0.004876355174928904, -0.007969613187015057,..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,"[-0.02760046347975731, -0.013719998300075531, ..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"[-0.0297758337110281, -0.014837449416518211, 0..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","[0.005073545966297388, 0.004160495940595865, 0..."


In [60]:
def list_ids_in_folder(folder_path):
    ids = set()
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            # Extracting the ID from the filename
            file_id = filename.split("_")[0][1:]
            ids.add(file_id)
    
    return list(ids)

In [64]:
folder_path = 'content_recommendations'
users_list = list_ids_in_folder(folder_path)
users_list = ['U' + num for num in users_list]
len(users_list)

3009

In [82]:
def category_correlation(method, n_recommend):
    
    all_pearson = []
    all_pvalue = []
    
    if method == "content":
        folder_path = 'content_recommendations'
    
    users_list = list_ids_in_folder(folder_path)
    users_list = ['U' + num for num in users_list]
    
    for user_id in users_list:
        if method == "content":
            item_path = "content_recommendations/" + user_id + "_content.csv"
        read_articles = interactions[interactions['User'] == user_id]['ID'].str.split().explode().tolist()
        user_news_df = news[news['ID'].isin(read_articles)]
        categ_counts_user = user_news_df['Category'].value_counts()
            
        recommendation_df = pd.read_csv(item_path)
        recpmmendations = recommendation_df.head(n_recommend)
        rec_list = recpmmendations['ID'].tolist() 
        user_news_rec = news[news['ID'].isin(rec_list)]
        recom_counts_user = user_news_rec['Category'].value_counts()
            
        merged_df = pd.concat([recom_counts_user, categ_counts_user], axis=1, sort=False).fillna(0)
        merged_df.columns = ['recommend', 'profiles']
        
        try:
            correlation, p_value = pearsonr(merged_df['recommend'], merged_df['profiles'])
            all_pearson.append(correlation)
            all_pvalue.append(p_value)
        except ValueError as e:
            print("An error occurred:", e)
            print("Skipping correlation calculation due to insufficient data.")
        
    print("Pearson correlation coefficient:", np.nanmean(all_pearson))
    print("P-value:", np.nanmean(all_pvalue))

In [84]:
category_correlation("content", 20)

An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation calculation due to insufficient data.
An error occurred: x and y must have length at least 2.
Skipping correlation cal

In [4]:
user_id = "U244"

In [12]:
recommendation_df = pd.read_csv("collaborative_recommendations/" + user_id + "_collab.csv")

In [41]:
read_articles = interactions[interactions['User'] == user_id]['ID'].str.split().explode().tolist()
user_news_df = news[news['ID'].isin(read_articles)]
categories_user = user_news_df['Category'].unique()
categ_counts_user = user_news_df['Category'].value_counts()

In [42]:
categ_counts_user

news            8
lifestyle       5
health          3
foodanddrink    3
tv              3
movies          2
autos           2
finance         2
video           1
Name: Category, dtype: int64

In [44]:
recpmmendations_20 = recommendation_df.head(20)
rec_list = recpmmendations_20['ID'].tolist() 
user_news_rec = news[news['ID'].isin(rec_list)]
recom_user = user_news_rec['Category'].unique()
recom_counts_user = user_news_rec['Category'].value_counts()

In [48]:
recom_counts_user

foodanddrink    7
news            7
sports          1
music           1
travel          1
lifestyle       1
finance         1
health          1
Name: Category, dtype: int64

In [54]:
merged_df = pd.concat([recom_counts_user, categ_counts_user], axis=1, sort=False).fillna(0)


In [56]:
merged_df.columns = ['recommend', 'profiles']

In [57]:
merged_df

,recommend,profiles
foodanddrink,7.0,3.0
news,7.0,8.0
sports,1.0,0.0
music,1.0,0.0
travel,1.0,0.0
lifestyle,1.0,5.0
finance,1.0,2.0
health,1.0,3.0
tv,0.0,3.0
movies,0.0,2.0


In [59]:
# Calculating Pearson correlation coefficient and p-value
correlation, p_value = pearsonr(merged_df['recommend'], merged_df['profiles'])

print("Pearson correlation coefficient:", correlation)
print("P-value:", p_value)

Pearson correlation coefficient: 0.599253180257948
P-value: 0.03947208893764183
